In [1]:
! pip install googletrans==4.0.0-rc1
! pip install nlpaug

In [5]:
! pip install nlpaug

     |████████████████████████████████| 410 kB 5.1 MB/s            


--------------------------------------------------------

# RD

In [1]:
import pandas as pd
import random

# RD(Random Deletion) 함수
def random_deletion_with_newlines(text, p=0.3):
    """
    줄바꿈(\n)이 포함된 문장에 대해 RD(Random Deletion)를 적용.

    Parameters:
    - text: 입력 문장 (str) - 여러 줄이 포함된 텍스트
    - p: 단어 삭제 확률 (float, 0.0 ~ 1.0)

    Returns:
    - 증강된 문장 (str) - 줄바꿈 형식을 유지한 결과
    """
    lines = text.split("\n")  # 텍스트를 줄 단위로 분리
    processed_lines = []  # 처리된 줄을 저장할 리스트

    for line in lines:
        words = line.split()  # 줄을 단어 단위로 분리
        if len(words) <= 1:  # 단어가 하나 이하일 경우 삭제하지 않음
            processed_lines.append(line)
            continue
        
        # 단어를 RD 기법으로 처리
        retained_words = [word for word in words if random.random() > p]
        if len(retained_words) == 0:  # 모든 단어가 삭제된 경우 최소 하나의 단어 유지
            retained_words = [random.choice(words)]
        
        # RD가 적용된 줄을 추가
        processed_lines.append(" ".join(retained_words))
    
    # 줄바꿈 기호를 다시 연결하여 반환
    return "\n".join(processed_lines)

# 데이터 증강 함수
def augment_data_with_rd(data, text_column, class_column, p=0.3):
    """
    RD(Random Deletion)를 사용하여 원본 데이터프레임을 증강. 원본 데이터와 동일한 구조로 반환.

    Parameters:
    - data: 입력 데이터프레임 (pd.DataFrame)
    - text_column: 텍스트가 포함된 열 이름 (str)
    - class_column: 클래스가 포함된 열 이름 (str)
    - p: RD(Random Deletion) 삭제 확률 (float, 0.0 ~ 1.0)

    Returns:
    - 증강된 데이터프레임 (pd.DataFrame)
    """
    augmented_rows = []  # 증강된 데이터 저장

    for _, row in data.iterrows():
        original_text = row[text_column]
        original_class = row[class_column]

        # RD 적용
        augmented_text = random_deletion_with_newlines(original_text, p=p)

        # 증강된 행 추가
        augmented_rows.append({class_column: original_class, text_column: augmented_text})

    # 증강 데이터프레임 생성
    augmented_df = pd.DataFrame(augmented_rows)
    return augmented_df

In [26]:
# train 파일 로드
file_path_train = "~/aiffel/AA_DLthon/data/train.csv"  # 사용하려는 CSV 파일 경로
df_train = pd.read_csv(file_path_train)

In [33]:
# 원본 데이터 개수 출력
original_count = len(df_train)
print(f"원본 데이터 수: {original_count}")

원본 데이터 수: 3950


In [27]:
print(df_train.columns)  # 열 이름 확인
print(len(df_train.columns))  # 열 개수 확인
df_train

Index(['idx', 'class', 'conversation'], dtype='object')
3


,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
3945,3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,3948,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [39]:
# RD 증강 수행
text_column = 'conversation'  # 증강할 텍스트 컬럼
class_column = 'class'  # 클래스 컬럼
p = 0.3  # 단어 삭제 확률

# 증강 데이터 생성
augmented_df_train = augment_data_with_rd(df_train, text_column=text_column, class_column=class_column, p=p)

# 원본 데이터에 구분 열 추가
df_train['data_type'] = '원본 데이터'

# 증강 데이터에 구분 열 추가
augmented_df_train['data_type'] = '증강 데이터'

# 원본 데이터와 증강 데이터 병합
combined_df_train = pd.concat([df_train, augmented_df_train], ignore_index=True)

# idx 열 재정렬
combined_df_train.reset_index(drop=True, inplace=True)
combined_df_train['idx'] = range(1, len(combined_df_train) + 1)

# 열 순서 재배치 (idx를 가장 앞으로)
columns_order = ['idx'] + [col for col in combined_df_train.columns if col != 'idx']
combined_df_train = combined_df_train[columns_order]

# 열 순서 재배치 (구분 열을 마지막에 위치)
columns_order = [col for col in combined_df_train.columns if col != 'data_type'] + ['data_type']
combined_df_train = combined_df_train[columns_order]

# 증강 데이터 개수 출력
augmented_count_train = len(combined_df_train)
print(f"증강된 데이터 수: {augmented_count_train}")

# 결과 출력
print("원본 데이터와 증강 데이터:")
print(combined_df_train)

# CSV 파일로 저장
output_file_path = "combined_rd_data.csv"
combined_df_train.to_csv(output_file_path, index=False, encoding="utf-8-sig")
print(f"결과가 {output_file_path}에 저장되었습니다.")

증강된 데이터 수: 7900
원본 데이터와 증강 데이터:
       idx        class                                       conversation  \
0        1        협박 대화  지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...   
1        2        협박 대화  길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...   
2        3    기타 괴롭힘 대화  너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...   
3        4        갈취 대화  어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...   
4        5        갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...   
...    ...          ...                                                ...   
7895  7896    기타 괴롭힘 대화  준하야 대가리가\n내 뭐.\n밥먹으면 대가리만 크냐\n머리만 큰다고 그러는거야\n주...   
7896  7897        갈취 대화  내가 지금 너 김길준 데리고 있어. 싶으면 계좌에 보내\n예.? 선생님 우리 말로 ...   
7897  7898  직장 내 괴롭힘 대화  나는 씨 사람 참 저렇게\n왜 들리겠어.\n들리라고 아니 일 못해 눈치 없어. 본인...   
7898  7899        갈취 대화  여기서\n이게\n남의 자릿세를 내야지.\n관청에 신고다하고 적법하게 장사하는 자릿세...   
7899  7900  직장 내 괴롭힘 대화  희정씨\n네?\n주말에 시간이 넘쳐나나봐\n갑자기\n손이 빤짝빤짝 네일했니?\n기분...   

     data_type  
0       원본 데이터

In [40]:
file_path_rd = "~/aiffel/AA_DLthon/combined_rd_data.csv"  # 사용하려는 CSV 파일 경로
df_rd = pd.read_csv(file_path_rd)
print(df_rd.columns)  # 열 이름 확인
print(len(df_rd.columns))  # 열 개수 확인
df_rd

Index(['idx', 'class', 'conversation', 'data_type'], dtype='object')
4


,idx,class,conversation,data_type
0,1,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,원본 데이터
1,2,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,원본 데이터
2,3,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,원본 데이터
3,4,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,원본 데이터
4,5,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,원본 데이터
...,...,...,...,...
7895,7896,기타 괴롭힘 대화,준하야 대가리가\n내 뭐.\n밥먹으면 대가리만 크냐\n머리만 큰다고 그러는거야\n주...,증강 데이터
7896,7897,갈취 대화,내가 지금 너 김길준 데리고 있어. 싶으면 계좌에 보내\n예.? 선생님 우리 말로 ...,증강 데이터
7897,7898,직장 내 괴롭힘 대화,나는 씨 사람 참 저렇게\n왜 들리겠어.\n들리라고 아니 일 못해 눈치 없어. 본인...,증강 데이터
7898,7899,갈취 대화,여기서\n이게\n남의 자릿세를 내야지.\n관청에 신고다하고 적법하게 장사하는 자릿세...,증강 데이터


-----

# RSW

In [8]:
import pandas as pd
import random

# RSW(Random Swap) 함수
def random_swap_with_newlines(text, num_swaps=1):
    """
    줄바꿈(\n)이 포함된 문장에 대해 RSW(Random Swap)를 적용.

    Parameters:
    - text: 입력 문장 (str) - 여러 줄이 포함된 텍스트
    - num_swaps: 교환할 단어 쌍의 개수 (int)

    Returns:
    - 증강된 문장 (str) - 줄바꿈 형식을 유지한 결과
    """
    lines = text.split("\n")  # 텍스트를 줄 단위로 분리
    processed_lines = []  # 처리된 줄을 저장할 리스트

    for line in lines:
        words = line.split()  # 줄을 단어 단위로 분리
        if len(words) <= 1:  # 단어가 하나 이하일 경우 스왑하지 않음
            processed_lines.append(line)
            continue

        # 단어를 RSW 기법으로 처리
        for _ in range(num_swaps):
            idx1, idx2 = random.sample(range(len(words)), 2)  # 무작위로 두 인덱스 선택
            words[idx1], words[idx2] = words[idx2], words[idx1]  # 단어 위치 교환

        # RSW가 적용된 줄을 추가
        processed_lines.append(" ".join(words))

    # 줄바꿈 기호를 다시 연결하여 반환
    return "\n".join(processed_lines)

# 데이터 증강 함수
def augment_data_with_rsw(data, text_column, class_column, num_swaps=1):
    """
    RSW(Random Swap)를 사용하여 원본 데이터프레임을 증강. 원본 데이터와 동일한 구조로 반환.

    Parameters:
    - data: 입력 데이터프레임 (pd.DataFrame)
    - text_column: 텍스트가 포함된 열 이름 (str)
    - class_column: 클래스가 포함된 열 이름 (str)
    - num_swaps: 교환할 단어 쌍의 개수 (int)

    Returns:
    - 증강된 데이터프레임 (pd.DataFrame)
    """
    augmented_rows = []  # 증강된 데이터 저장

    for _, row in data.iterrows():
        original_text = row[text_column]
        original_class = row[class_column]

        # RSW 적용
        augmented_text = random_swap_with_newlines(original_text, num_swaps=num_swaps)

        # 증강된 행 추가
        augmented_rows.append({class_column: original_class, text_column: augmented_text})

    # 증강 데이터프레임 생성
    augmented_df = pd.DataFrame(augmented_rows)
    return augmented_df

In [14]:
# train 파일 로드
file_path_train = "~/aiffel/AA_DLthon/data/train.csv"  # 사용하려는 CSV 파일 경로
df_train = pd.read_csv(file_path_train)

In [15]:
# 원본 데이터 개수 출력
original_count = len(df_train)
print(f"원본 데이터 수: {original_count}")

원본 데이터 수: 3950


In [16]:
print(df_train.columns)  # 열 이름 확인
print(len(df_train.columns))  # 열 개수 확인
df_train

Index(['idx', 'class', 'conversation'], dtype='object')
3


,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
3945,3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,3948,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [17]:
# RSW 증강 수행
text_column = 'conversation'  # 증강할 텍스트 컬럼
class_column = 'class'  # 클래스 컬럼
num_swaps = 2  # 각 문장에서 교환할 단어 쌍의 개수

# 증강 데이터 생성
augmented_df_train = augment_data_with_rsw(df_train, text_column=text_column, class_column=class_column, num_swaps=num_swaps)

# 원본 데이터에 구분 열 추가
df_train['data_type'] = '원본 데이터'

# 증강 데이터에 구분 열 추가
augmented_df_train['data_type'] = '증강 데이터'

# 원본 데이터와 증강 데이터 병합
combined_df_train = pd.concat([df_train, augmented_df_train], ignore_index=True)

# idx 열 재정렬
combined_df_train.reset_index(drop=True, inplace=True)
combined_df_train['idx'] = range(1, len(combined_df_train) + 1)

# 열 순서 재배치 (idx를 가장 앞으로)
columns_order = ['idx'] + [col for col in combined_df_train.columns if col != 'idx']
combined_df_train = combined_df_train[columns_order]

# 열 순서 재배치 (구분 열을 마지막에 위치)
columns_order = [col for col in combined_df_train.columns if col != 'data_type'] + ['data_type']
combined_df_train = combined_df_train[columns_order]

# 증강 데이터 개수 출력
augmented_count_train = len(combined_df_train)
print(f"증강된 데이터 수: {augmented_count_train}")

# 결과 출력
print("원본 데이터와 증강 데이터:")
print(combined_df_train)

# CSV 파일로 저장
output_file_path = "combined_rsw_data.csv"
combined_df_train.to_csv(output_file_path, index=False, encoding="utf-8-sig")
print(f"결과가 {output_file_path}에 저장되었습니다.")

증강된 데이터 수: 7900
원본 데이터와 증강 데이터:
       idx        class                                       conversation  \
0        1        협박 대화  지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...   
1        2        협박 대화  길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...   
2        3    기타 괴롭힘 대화  너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...   
3        4        갈취 대화  어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...   
4        5        갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...   
...    ...          ...                                                ...   
7895  7896    기타 괴롭힘 대화  준하야 대가리가 왜이렇게 넌 크냐?\n머리가 뭐. 내\n대가리만 너는? 크냐 밥먹으...   
7896  7897        갈취 대화  1억만 지금 너 계좌에 김길준 데리고 있어. 살리고 싶으면 아들 내가 보내\n우리 ...   
7897  7898  직장 내 괴롭힘 대화  어떻게 씨 같은 사람 보면 참 저렇게 나는 신기하더라. 살지.\n들리겠어. 왜 그래...   
7898  7899        갈취 대화  여기서 장사하래? 누구맘대로\n이게 무슨일입니까?\n장사하려면 구역에서 내야지. 자...   
7899  7900  직장 내 괴롭힘 대화  희정씨\n네?\n넘쳐나나봐 주말에 시간이\n갑자기 왜그러세요?\n빤짝빤짝 네일했니?...   

     data_type  
0       원본 데이터

In [18]:
file_path_rsw = "~/aiffel/AA_DLthon/combined_rsw_data.csv"  # 사용하려는 CSV 파일 경로
df_rsw = pd.read_csv(file_path_rsw)
print(df_rsw.columns)  # 열 이름 확인
print(len(df_rsw.columns))  # 열 개수 확인
df_rsw

Index(['idx', 'class', 'conversation', 'data_type'], dtype='object')
4


,idx,class,conversation,data_type
0,1,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,원본 데이터
1,2,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,원본 데이터
2,3,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,원본 데이터
3,4,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,원본 데이터
4,5,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,원본 데이터
...,...,...,...,...
7895,7896,기타 괴롭힘 대화,준하야 대가리가 왜이렇게 넌 크냐?\n머리가 뭐. 내\n대가리만 너는? 크냐 밥먹으...,증강 데이터
7896,7897,갈취 대화,1억만 지금 너 계좌에 김길준 데리고 있어. 살리고 싶으면 아들 내가 보내\n우리 ...,증강 데이터
7897,7898,직장 내 괴롭힘 대화,어떻게 씨 같은 사람 보면 참 저렇게 나는 신기하더라. 살지.\n들리겠어. 왜 그래...,증강 데이터
7898,7899,갈취 대화,여기서 장사하래? 누구맘대로\n이게 무슨일입니까?\n장사하려면 구역에서 내야지. 자...,증강 데이터


--------------------

# 하프 데이터

In [20]:
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
file_path = '~/aiffel/AA_DLthon/data/train.csv'  # 적절한 경로로 변경
df_train = pd.read_csv(file_path)

# 클래스별 데이터 개수 계산
class_counts = df_train['class'].value_counts()

# 각 클래스의 개수를 숫자로 출력
print("Class-wise Sample Counts:")
for cls, count in class_counts.items():
    print(f"{cls}: {count}")

Class-wise Sample Counts:
기타 괴롭힘 대화: 1094
갈취 대화: 981
직장 내 괴롭힘 대화: 979
협박 대화: 896


In [22]:
import pandas as pd

# 데이터 로드
file_path = '~/aiffel/AA_DLthon/data/train.csv'  # 파일 경로를 실제 데이터 파일 경로로 수정
df_train = pd.read_csv(file_path)

# 결과를 저장할 데이터프레임 초기화
sampled_data = pd.DataFrame()

# 각 클래스별로 절반씩 샘플링
for cls in df_train['class'].unique():
    class_subset = df_train[df_train['class'] == cls]
    sampled_subset = class_subset.sample(frac=0.5, random_state=42)  # 절반 샘플링
    sampled_data = pd.concat([sampled_data, sampled_subset], ignore_index=True)

# 샘플링된 데이터를 CSV로 저장
output_file_path = 'sampled_half_train.csv'
sampled_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

# 결과 출력
print(f"샘플링된 데이터가 {output_file_path}에 저장되었습니다.")

샘플링된 데이터가 sampled_half_train.csv에 저장되었습니다.


In [2]:
file_half = "~/aiffel/AA_DLthon/sampled_half_train.csv"  # 사용하려는 CSV 파일 경로
df_half = pd.read_csv(file_half)

original_half = len(df_half)
print(f"원본 데이터 수: {original_half}")

print(df_half.columns)  # 열 이름 확인
print(len(df_half.columns))  # 열 개수 확인
df_half

원본 데이터 수: 1975
Index(['idx', 'class', 'conversation'], dtype='object')
3


,idx,class,conversation
0,3222,협박 대화,이게 누구게\n 그만하시죠\n봐라봐라 바로 알아보네 맞다 니 동생 이쁘게 생겼더라 ...
1,1935,협박 대화,제대로 안하지?\n일단 미안해 사과할게\n필요없고 오늘 안으로 보내\n혹시 내일까지...
2,2336,협박 대화,음식 리뷰를 그렇게 쓰면 어떻합니까? 당장 글 삭제해!\n 솔직하게 음식먹은 내용을...
3,3251,협박 대화,너 그 여자 좋아하지? 언제부터야?\n아닌데? 그리고 우리 헤어졌는데 뭔 상관이야\...
4,210,협박 대화,내가 돈 훔쳤다고 니가 고발했지?\n 나 아니야 왜그래 그리고 니가 훔친거 맞잖아\...
...,...,...,...
1970,2161,직장 내 괴롭힘 대화,여봐요 씨\n네 과장님!\n연차를 쓸거면 육하원칙에 맞게.써야할거 아니야!!!\n예...
1971,1420,직장 내 괴롭힘 대화,정희씨 내가 시킨 일은 다 했어?\n아니요 잠시 다른 일좀\n무슨일?\n기획팀에서 ...
1972,2452,직장 내 괴롭힘 대화,지연씨 내일에 무슨 약속 있어요?\n 아니요 없습니다.\n그래요? 잘됐네. 내일 연...
1973,83,직장 내 괴롭힘 대화,하사원 여기 놀러왓냐\n아닙니다\n이것밖에 못햇냐\n죄송해요\n월급만 축내네 진짜\...


In [3]:
# RD 증강 수행
text_column = 'conversation'  # 증강할 텍스트 컬럼
class_column = 'class'  # 클래스 컬럼
p = 0.3  # 단어 삭제 확률

# 증강 데이터 생성
augmented_df_half = augment_data_with_rd(df_half, text_column=text_column, class_column=class_column, p=p)

# 증강 데이터에 구분 열 추가
augmented_df_half['data_type'] = '증강 데이터'

# idx 열 추가 (1부터 시작)
augmented_df_half.reset_index(drop=True, inplace=True)
augmented_df_half['idx'] = range(1, len(augmented_df_half) + 1)

# 열 순서 재배치 (idx를 가장 앞으로)
columns_order = ['idx'] + [col for col in augmented_df_half.columns if col != 'idx']
augmented_df_half = augmented_df_half[columns_order]

# 열 순서 재배치 (구분 열을 마지막에 위치)
columns_order = [col for col in augmented_df_half.columns if col != 'data_type'] + ['data_type']
augmented_df_half = augmented_df_half[columns_order]

# 증강 데이터 개수 출력
augmented_count_half = len(augmented_df_half)
print(f"증강된 데이터 수: {augmented_count_half}")

# 결과 출력
print("증강 데이터:")
print(augmented_df_half)

# 증강 데이터만 CSV로 저장
output_file_path = "augmented_rd_half_data.csv"
augmented_df_half.to_csv(output_file_path, index=False, encoding="utf-8-sig")
print(f"증강 데이터가 {output_file_path}에 저장되었습니다.")

증강된 데이터 수: 1975
증강 데이터:
       idx        class                                       conversation  \
0        1        협박 대화  이게 누구게\n 그만하시죠\n봐라봐라 바로 맞다 니 동생 생겼더라 ?\n제동생은 얘...   
1        2        협박 대화  제대로 안하지?\n일단 미안해 사과할게\n필요없고\n기다려 수 미안한데\n친구 원하...   
2        3        협박 대화  리뷰를 그렇게 쓰면 당장 글\n솔직하게 음식먹은 내용을 리뷰한건데요?\n누구 장사 ...   
3        4        협박 대화  너 그 여자 좋아하지? 언제부터야?\n그리고 우리 뭔\n다른 여자한테 플러팅하고 내...   
4        5        협박 대화  돈 훔쳤다고 니가\n나 아니야 그리고 훔친거 맞잖아\n내가 증거있어??\n내가 보고...   
...    ...          ...                                                ...   
1970  1971  직장 내 괴롭힘 대화  씨\n네 과장님!\n연차를 쓸거면 육하원칙에\n예?\n몰라요? 언제 어디서 왜 무엇...   
1971  1972  직장 내 괴롭힘 대화  정희씨 내가 시킨 했어?\n아니요 잠시 일좀\n무슨일?\n기획팀에서 마지막 확인좀 ...   
1972  1973  직장 내 괴롭힘 대화  지연씨 내일에 약속 있어요?\n없습니다.\n잘됐네. 내일 연차 나랑\n때 다녀올 생...   
1973  1974  직장 내 괴롭힘 대화  하사원 여기 놀러왓냐\n아닙니다\n이것밖에 못햇냐\n죄송해요\n월급만 축내네 진짜\...   
1974  1975  직장 내 괴롭힘 대화  나씨\n싫습니다. 저만 것도 아닌데 저만\n나씨가 저희 부서 유일한\n저 일꾼 아니...   

     data_type  
0       증강 데이터  
1    

In [5]:
file_path_rd_half = "~/aiffel/AA_DLthon/augmented_rd_half_data.csv"  # 사용하려는 CSV 파일 경로
df_half = pd.read_csv(file_path_rd_half)
print(df_half.columns)  # 열 이름 확인
print(len(df_half.columns))  # 열 개수 확인
df_half

Index(['idx', 'class', 'conversation', 'data_type'], dtype='object')
4


,idx,class,conversation,data_type
0,1,협박 대화,이게 누구게\n 그만하시죠\n봐라봐라 바로 맞다 니 동생 생겼더라 ?\n제동생은 얘...,증강 데이터
1,2,협박 대화,제대로 안하지?\n일단 미안해 사과할게\n필요없고\n기다려 수 미안한데\n친구 원하...,증강 데이터
2,3,협박 대화,리뷰를 그렇게 쓰면 당장 글\n솔직하게 음식먹은 내용을 리뷰한건데요?\n누구 장사 ...,증강 데이터
3,4,협박 대화,너 그 여자 좋아하지? 언제부터야?\n그리고 우리 뭔\n다른 여자한테 플러팅하고 내...,증강 데이터
4,5,협박 대화,돈 훔쳤다고 니가\n나 아니야 그리고 훔친거 맞잖아\n내가 증거있어??\n내가 보고...,증강 데이터
...,...,...,...,...
1970,1971,직장 내 괴롭힘 대화,씨\n네 과장님!\n연차를 쓸거면 육하원칙에\n예?\n몰라요? 언제 어디서 왜 무엇...,증강 데이터
1971,1972,직장 내 괴롭힘 대화,정희씨 내가 시킨 했어?\n아니요 잠시 일좀\n무슨일?\n기획팀에서 마지막 확인좀 ...,증강 데이터
1972,1973,직장 내 괴롭힘 대화,지연씨 내일에 약속 있어요?\n없습니다.\n잘됐네. 내일 연차 나랑\n때 다녀올 생...,증강 데이터
1973,1974,직장 내 괴롭힘 대화,하사원 여기 놀러왓냐\n아닙니다\n이것밖에 못햇냐\n죄송해요\n월급만 축내네 진짜\...,증강 데이터


In [11]:
# RSW 증강 수행
text_column = 'conversation'  # 증강할 텍스트 컬럼
class_column = 'class'  # 클래스 컬럼
num_swaps = 2  # 각 문장에서 교환할 단어 쌍의 개수

# 증강 데이터 생성
augmented_df_half = augment_data_with_rsw(df_half, text_column=text_column, class_column=class_column, num_swaps=num_swaps)

# 증강 데이터에 구분 열 추가
augmented_df_half['data_type'] = '증강 데이터'

# idx 열 재정렬
augmented_df_half.reset_index(drop=True, inplace=True)
augmented_df_half['idx'] = range(1, len(augmented_df_half) + 1)

# 열 순서 재배치 (idx를 가장 앞으로)
columns_order = ['idx'] + [col for col in augmented_df_half.columns if col != 'idx']
augmented_df_half = augmented_df_half[columns_order]

# 열 순서 재배치 (구분 열을 마지막에 위치)
columns_order = [col for col in augmented_df_half.columns if col != 'data_type'] + ['data_type']
augmented_df_half = augmented_df_half[columns_order]

# 증강 데이터 개수 출력
augmented_count_half = len(augmented_df_half)
print(f"증강된 데이터 수: {augmented_count_half}")

# 결과 출력
print("증강 데이터:")
print(augmented_df_half)

# CSV 파일로 저장
output_file_path = "augmented_rsw_half_data.csv"
augmented_df_half.to_csv(output_file_path, index=False, encoding="utf-8-sig")
print(f"결과가 {output_file_path}에 저장되었습니다.")

증강된 데이터 수: 1975
증강 데이터:
       idx        class                                       conversation  \
0        1        협박 대화  이게 누구게\n 그만하시죠\n동생 맞다 바로 니 봐라봐라 생겼더라 ?\n제동생은 얘...   
1        2        협박 대화  제대로 안하지?\n사과할게 일단 미안해\n필요없고\n기다려 수 미안한데\n친구 원하...   
2        3        협박 대화  당장 글 쓰면 리뷰를 그렇게\n음식먹은 솔직하게 리뷰한건데요? 내용을\n하는거야? ...   
3        4        협박 대화  언제부터야? 그 너 좋아하지? 여자\n뭔 그리고 우리\n심지어 여자한테 플러팅하고 ...   
4        5        협박 대화  훔쳤다고 니가 돈\n나 맞잖아 아니야 훔친거 그리고\n내가 증거있어??\n내가 보고...   
...    ...          ...                                                ...   
1970  1971  직장 내 괴롭힘 대화  씨\n네 과장님!\n연차를 쓸거면 육하원칙에\n예?\n언제 몰라요? 어디서 써야할거...   
1971  1972  직장 내 괴롭힘 대화  했어? 정희씨 시킨 내가\n아니요 잠시 일좀\n무슨일?\n부탁해서요 기획팀에서 확인...   
1972  1973  직장 내 괴롭힘 대화  있어요? 내일에 지연씨 약속\n없습니다.\n잘됐네. 연차 나랑 내일\n때 다녀올 생...   
1973  1974  직장 내 괴롭힘 대화  놀러왓냐 하사원 여기\n아닙니다\n이것밖에 못햇냐\n죄송해요\n월급만 축내네 진짜\...   
1974  1975  직장 내 괴롭힘 대화  나씨\n아닌데 저만 저만 싫습니다. 것도\n나씨가 유일한 저희 부서\n아니고 일꾼 ...   

     data_type  
0       증강 데이터  
1    

In [12]:
file_path_rd_half = "~/aiffel/AA_DLthon/augmented_rsw_half_data.csv"  # 사용하려는 CSV 파일 경로
df_half = pd.read_csv(file_path_rd_half)
print(df_half.columns)  # 열 이름 확인
print(len(df_half.columns))  # 열 개수 확인
df_half

Index(['idx', 'class', 'conversation', 'data_type'], dtype='object')
4


,idx,class,conversation,data_type
0,1,협박 대화,이게 누구게\n 그만하시죠\n동생 맞다 바로 니 봐라봐라 생겼더라 ?\n제동생은 얘...,증강 데이터
1,2,협박 대화,제대로 안하지?\n사과할게 일단 미안해\n필요없고\n기다려 수 미안한데\n친구 원하...,증강 데이터
2,3,협박 대화,당장 글 쓰면 리뷰를 그렇게\n음식먹은 솔직하게 리뷰한건데요? 내용을\n하는거야? ...,증강 데이터
3,4,협박 대화,언제부터야? 그 너 좋아하지? 여자\n뭔 그리고 우리\n심지어 여자한테 플러팅하고 ...,증강 데이터
4,5,협박 대화,훔쳤다고 니가 돈\n나 맞잖아 아니야 훔친거 그리고\n내가 증거있어??\n내가 보고...,증강 데이터
...,...,...,...,...
1970,1971,직장 내 괴롭힘 대화,씨\n네 과장님!\n연차를 쓸거면 육하원칙에\n예?\n언제 몰라요? 어디서 써야할거...,증강 데이터
1971,1972,직장 내 괴롭힘 대화,했어? 정희씨 시킨 내가\n아니요 잠시 일좀\n무슨일?\n부탁해서요 기획팀에서 확인...,증강 데이터
1972,1973,직장 내 괴롭힘 대화,있어요? 내일에 지연씨 약속\n없습니다.\n잘됐네. 연차 나랑 내일\n때 다녀올 생...,증강 데이터
1973,1974,직장 내 괴롭힘 대화,놀러왓냐 하사원 여기\n아닙니다\n이것밖에 못햇냐\n죄송해요\n월급만 축내네 진짜\...,증강 데이터


In [13]:
import pandas as pd

# 파일 경로 설정
rd_file_path = "augmented_rd_half_data.csv"
rsw_file_path = "augmented_rsw_half_data.csv"

# 두 파일 불러오기
augmented_rd_df = pd.read_csv(rd_file_path)
augmented_rsw_df = pd.read_csv(rsw_file_path)

# 두 데이터프레임 병합 후 중복 제거
combined_augmented_df = pd.concat([augmented_rd_df, augmented_rsw_df], ignore_index=True).drop_duplicates()

# idx 열 재정렬
combined_augmented_df['idx'] = range(1, len(combined_augmented_df) + 1)

# 열 순서 재배치 (idx를 가장 앞으로)
columns_order = ['idx'] + [col for col in combined_augmented_df.columns if col != 'idx']
combined_augmented_df = combined_augmented_df[columns_order]

# CSV 파일로 저장
output_file_path = "combined_augmented_half_data.csv"
combined_augmented_df.to_csv(output_file_path, index=False, encoding="utf-8-sig")

# 결과 출력
print(f"중복을 제거한 증강 데이터가 {output_file_path}에 저장되었습니다.")

중복을 제거한 증강 데이터가 combined_augmented_half_data.csv에 저장되었습니다.


In [14]:
file = "~/aiffel/AA_DLthon/combined_augmented_half_data.csv"  # 사용하려는 CSV 파일 경로
half = pd.read_csv(file)
print(half.columns)  # 열 이름 확인
print(len(half.columns))  # 열 개수 확인
half

Index(['idx', 'class', 'conversation', 'data_type'], dtype='object')
4


,idx,class,conversation,data_type
0,1,협박 대화,이게 누구게\n 그만하시죠\n봐라봐라 바로 맞다 니 동생 생겼더라 ?\n제동생은 얘...,증강 데이터
1,2,협박 대화,제대로 안하지?\n일단 미안해 사과할게\n필요없고\n기다려 수 미안한데\n친구 원하...,증강 데이터
2,3,협박 대화,리뷰를 그렇게 쓰면 당장 글\n솔직하게 음식먹은 내용을 리뷰한건데요?\n누구 장사 ...,증강 데이터
3,4,협박 대화,너 그 여자 좋아하지? 언제부터야?\n그리고 우리 뭔\n다른 여자한테 플러팅하고 내...,증강 데이터
4,5,협박 대화,돈 훔쳤다고 니가\n나 아니야 그리고 훔친거 맞잖아\n내가 증거있어??\n내가 보고...,증강 데이터
...,...,...,...,...
3925,3926,직장 내 괴롭힘 대화,씨\n네 과장님!\n연차를 쓸거면 육하원칙에\n예?\n언제 몰라요? 어디서 써야할거...,증강 데이터
3926,3927,직장 내 괴롭힘 대화,했어? 정희씨 시킨 내가\n아니요 잠시 일좀\n무슨일?\n부탁해서요 기획팀에서 확인...,증강 데이터
3927,3928,직장 내 괴롭힘 대화,있어요? 내일에 지연씨 약속\n없습니다.\n잘됐네. 연차 나랑 내일\n때 다녀올 생...,증강 데이터
3928,3929,직장 내 괴롭힘 대화,놀러왓냐 하사원 여기\n아닙니다\n이것밖에 못햇냐\n죄송해요\n월급만 축내네 진짜\...,증강 데이터


In [15]:
import pandas as pd

# 파일 경로 설정
rd_file_path = "combined_rd_data.csv"
rsw_file_path = "combined_rsw_data.csv"
augmented_half_file_path = "combined_augmented_half_data.csv"

# 파일 불러오기
combined_rd_df = pd.read_csv(rd_file_path)
combined_rsw_df = pd.read_csv(rsw_file_path)
combined_augmented_half_df = pd.read_csv(augmented_half_file_path)

# 중복 제거를 위해 원본 데이터 구분
rd_original = combined_rd_df[combined_rd_df['data_type'] == '원본 데이터']
rsw_original = combined_rsw_df[combined_rsw_df['data_type'] == '원본 데이터']

# combined_rsw_data에서 원본 데이터 제거
combined_rsw_df = combined_rsw_df[combined_rsw_df['data_type'] != '원본 데이터']

# 세 데이터프레임 병합
combined_df = pd.concat([rd_original, combined_rsw_df, combined_augmented_half_df], ignore_index=True)

# 데이터 개수 조정 (12,000개로 맞춤)
if len(combined_df) > 12000:
    combined_df = combined_df.sample(n=12000, random_state=42).reset_index(drop=True)

# idx 열 재정렬
combined_df['idx'] = range(1, len(combined_df) + 1)

# 열 순서 재배치 (idx를 가장 앞으로)
columns_order = ['idx'] + [col for col in combined_df.columns if col != 'idx']
combined_df = combined_df[columns_order]

# CSV 파일로 저장
output_file_path = "final_combined_data.csv"
combined_df.to_csv(output_file_path, index=False, encoding="utf-8-sig")

# 결과 출력
print(f"최종 데이터가 {output_file_path}에 저장되었습니다. 총 데이터 수: {len(combined_df)}")

최종 데이터가 final_combined_data.csv에 저장되었습니다. 총 데이터 수: 11830


In [16]:
file_final = "~/aiffel/AA_DLthon/final_combined_data.csv"  # 사용하려는 CSV 파일 경로
final = pd.read_csv(file_final)
print(final.columns)  # 열 이름 확인
print(len(final.columns))  # 열 개수 확인
final

Index(['idx', 'class', 'conversation', 'data_type'], dtype='object')
4


,idx,class,conversation,data_type
0,1,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,원본 데이터
1,2,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,원본 데이터
2,3,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,원본 데이터
3,4,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,원본 데이터
4,5,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,원본 데이터
...,...,...,...,...
11825,11826,직장 내 괴롭힘 대화,씨\n네 과장님!\n연차를 쓸거면 육하원칙에\n예?\n언제 몰라요? 어디서 써야할거...,증강 데이터
11826,11827,직장 내 괴롭힘 대화,했어? 정희씨 시킨 내가\n아니요 잠시 일좀\n무슨일?\n부탁해서요 기획팀에서 확인...,증강 데이터
11827,11828,직장 내 괴롭힘 대화,있어요? 내일에 지연씨 약속\n없습니다.\n잘됐네. 연차 나랑 내일\n때 다녀올 생...,증강 데이터
11828,11829,직장 내 괴롭힘 대화,놀러왓냐 하사원 여기\n아닙니다\n이것밖에 못햇냐\n죄송해요\n월급만 축내네 진짜\...,증강 데이터
